In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2017-2021_restaurant(2)_reviews.csv')

In [ ]:
df.head()

,business_id,name,categories,avg_stars,review_id,user_id,stars,useful,funny,cool,text,date,Mexican,count
0,-ATiAtTikuGuqvaW2O6tNA,Marathon,"Sandwiches, American (New), Lounges, Nightlife...",3.5,7FLXp3VCE1RHOqsGBk2JNg,OBoeuSW39_uwUXGtN4vdOA,4,0,0,0,"Straight to the point: the service, the one di...",2017-09-09 12:33:12,False,113
1,-ATiAtTikuGuqvaW2O6tNA,Marathon,"Sandwiches, American (New), Lounges, Nightlife...",3.5,DGkj-4PdsCtzQUtN5A42tA,mhwNRi3mBekFM-9zN2aRxw,5,0,0,0,Great turkey burger. I would've taken a photo ...,2017-11-10 18:48:06,False,113
2,-ATiAtTikuGuqvaW2O6tNA,Marathon,"Sandwiches, American (New), Lounges, Nightlife...",3.5,ulLwjsg4vc-tjsLI3lx6QQ,PWjXfHHm5LQhyPgZcedLzA,3,0,0,0,I ordered the turkey chilli and it was sold ou...,2018-03-19 11:13:32,False,113
3,-ATiAtTikuGuqvaW2O6tNA,Marathon,"Sandwiches, American (New), Lounges, Nightlife...",3.5,29pECaECmT2iHVDK_gY8bA,s2apTe1qD9Bytunpj4enMw,4,0,0,0,Went here with friends on a Thursday evening a...,2018-07-18 00:49:32,False,113
4,-ATiAtTikuGuqvaW2O6tNA,Marathon,"Sandwiches, American (New), Lounges, Nightlife...",3.5,jS-Se0swmLUwYsFjeNoLTg,QZy2N84_45OkPU_oUxy6Dw,4,0,0,0,Great place. We had excellent 4 star vegetaria...,2017-02-20 01:11:49,False,113


# Clean data and get rating data

In [ ]:
 #Get business_id, user_id, stars for recommender

recommender_df = df[['business_id', 'user_id', 'stars']]
recommender_df.head()

,business_id,user_id,stars
0,-ATiAtTikuGuqvaW2O6tNA,OBoeuSW39_uwUXGtN4vdOA,4
1,-ATiAtTikuGuqvaW2O6tNA,mhwNRi3mBekFM-9zN2aRxw,5
2,-ATiAtTikuGuqvaW2O6tNA,PWjXfHHm5LQhyPgZcedLzA,3
3,-ATiAtTikuGuqvaW2O6tNA,s2apTe1qD9Bytunpj4enMw,4
4,-ATiAtTikuGuqvaW2O6tNA,QZy2N84_45OkPU_oUxy6Dw,4


In [ ]:
reviews_count_df = recommender_df.groupby('user_id')['stars'].count()
reviews_count_df.head(5)

user_id
--S8M395r8NtOCvS2LRfDw    1
-073IXD_JkLK8SlRcsV-og    1
-0TIxezdO7xXrkP7zXFd8Q    1
-0ULpQ_pBrG2wOFOG2Iejg    1
-0aInSHjCWLfiNqfgmWnow    1
Name: stars, dtype: int64

In [ ]:
print('Max reviews: %s, Min reviews: %s' % (max(reviews_count_df), min(reviews_count_df)))
print('Median reviews: %s, Mean reviews: %s' % (np.median(reviews_count_df), round(np.mean(reviews_count_df),2)))
print('25%% reviews: %d,  75%% reviews: %d' % (np.percentile(reviews_count_df, 25), np.percentile(reviews_count_df, 75)))
print('Number of unique business: %d' % (len(set(recommender_df['business_id']))))

Max reviews: 21, Min reviews: 1
Median reviews: 1.0, Mean reviews: 1.22
25% reviews: 1,  75% reviews: 1
Number of unique business: 445


In [ ]:
active_user = list(reviews_count_df[reviews_count_df >= 5].index)
mask = [user in active_user for user in recommender_df['user_id']]
active_user_df = recommender_df[mask]
active_user_df.head(5)

,business_id,user_id,stars
28,-ATiAtTikuGuqvaW2O6tNA,Zq31drx-JM2R1MKQg8uJQw,4
50,-ATiAtTikuGuqvaW2O6tNA,UqTvUrtM9VZfcnjMyvfZxA,3
52,-ATiAtTikuGuqvaW2O6tNA,55fCofuKVb7XP3newzmX3w,4
58,-ATiAtTikuGuqvaW2O6tNA,l3ElIU_Wsz16qnBp690mvA,4
60,-ATiAtTikuGuqvaW2O6tNA,i9pHdGTNsm-Uh8dHDwKATw,3


In [ ]:
# reconstruct user id
from scipy import sparse
highest_user_id = len(set(active_user_df['user_id']))
highest_movie_id = len(set(active_user_df['business_id']))
ratings_mat = sparse.lil_matrix((highest_user_id, highest_movie_id))
ratings_mat

<104x208 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [ ]:
user_id = list(set(active_user_df['user_id']))
business_id = list(set(active_user_df['business_id']))
for _, row in active_user_df.iterrows():
    ratings_mat[user_id.index(row.user_id), business_id.index(row.business_id)] = row.stars
ratings_mat

<104x208 sparse matrix of type '<class 'numpy.float64'>'
	with 706 stored elements in List of Lists format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from time import time
class ItemItemRecommender(object):

    def __init__(self, neighborhood_size):
        self.neighborhood_size = neighborhood_size

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        self.item_sim_mat = cosine_similarity(self.ratings_mat.T)
        self._set_neighborhoods()

    def _set_neighborhoods(self):
        least_to_most_sim_indexes = np.argsort(self.item_sim_mat, 1)
        self.neighborhoods = least_to_most_sim_indexes[:, -self.neighborhood_size:]

    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        # Just initializing so I have somewhere to put rating preds
        out = np.zeros(self.n_items)
        for item_to_rate in range(self.n_items):
            relevant_items = np.intersect1d(self.neighborhoods[item_to_rate],
                                            items_rated_by_this_user,
                                            assume_unique=True)  # assume_unique speeds up intersection op
            out[item_to_rate] = self.ratings_mat[user_id, relevant_items] * \
                self.item_sim_mat[item_to_rate, relevant_items] / \
                self.item_sim_mat[item_to_rate, relevant_items].sum()
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        cleaned_out = np.nan_to_num(out)
        return cleaned_out

    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n):
        pred_ratings = self.pred_one_user(user_id)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:], items_rated_by_this_user

In [ ]:
my_rec_engine = ItemItemRecommender(neighborhood_size=80)
my_rec_engine.fit(ratings_mat)

In [ ]:
lucky_user = np.random.choice(active_user_df['user_id'], 1)[0]
lucky_user_index = user_id.index(lucky_user)
lucky_user_recommend, items_rated_by_this_user = my_rec_engine.top_n_recs(user_id=lucky_user_index, n = 10)

<ipython-input-10-92709d5a8ec5>:28: RuntimeWarning: invalid value encountered in true_divide
  out[item_to_rate] = self.ratings_mat[user_id, relevant_items] * \


In [ ]:
print("The top ten recommendation for user %s are: " % (lucky_user))
print('%s' % (', '.join(list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                       for i in lucky_user_recommend)))

The top ten recommendation for user OLGQ7alK4VKl3YdQk6UF5g are: 
Tria Cafe Wash West, Marrakesh, In Riva, Mumbai Bistro, Suraya, Han Dynasty, Wiz Kid, Octopus Falafel Truck, HipCityVeg - University City, Green Basil Thai Kitchen


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
original_rated_restaurants = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] for i in items_rated_by_this_user]
mask = [name in original_rated_restaurants for name in df['name']]
original_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
original_category_vec = vectorizer.fit_transform(original_category).toarray()
original_word = vectorizer.get_feature_names_out()
print('Categories from user rated restaurants: \n%s' % (','.join(i for i in original_word)))

Categories from user rated restaurants: 
american,automotive,bar,bars,beer,breakfast,brunch,burgers,cocktail,coffee,convenience,cuban,delis,food,gas,gastropubs,italian,lounges,new,nightlife,plates,restaurants,sandwiches,seafood,small,stations,steakhouses,stores,tapas,tea,traditional,whiskey


In [ ]:
recommend_res = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                 for i in lucky_user_recommend]
mask = [name in recommend_res for name in df['name']]
recommend_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
recommend_category_vec = vectorizer.fit_transform(recommend_category).toarray()
recommend_word = vectorizer.get_feature_names_out()
print('Categories from recommend restaurants: \n%s' % (','.join(i for i in recommend_word)))

Categories from recommend restaurants: 
active,american,bars,beer,burgers,cafes,cheese,cheesesteaks,chinese,cocktail,coffee,desserts,eastern,european,falafel,fast,food,gardens,grocery,indian,italian,lebanese,life,mediterranean,middle,modern,moroccan,new,nightlife,parks,pizza,restaurants,sandwiches,shops,specialty,spirits,stands,szechuan,tea,thai,trucks,vegan,wine


In [ ]:
#Check the common labels
print("Common labels are: \n%s" % (', '.join(word for word in recommend_word if word in original_word)))

Common labels are: 
american, bars, beer, burgers, cocktail, coffee, food, italian, new, nightlife, restaurants, sandwiches, tea


# Matrix Factorization recommender (NMF)

In [ ]:
from sklearn.decomposition import NMF
class NMF_Recommender(object):

    def __init__(self, n_components):
        self.n_components = n_components

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        nmf = NMF(n_components = 200)
        nmf.fit(ratings_mat)
        self.W = nmf.transform(ratings_mat)
        self.H = nmf.components_
        self.error = nmf.reconstruction_err_
        self.ratings_mat_fitted = self.W.dot(self.H)

    def get_error(self):
        return self.error
        
    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        cleaned_out = self.ratings_mat_fitted[user_id,:]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return cleaned_out

    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n):
        pred_ratings = self.pred_one_user(user_id)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:], items_rated_by_this_user

In [ ]:

# get recommendations for the same lucky user
my_rec_engine = NMF_Recommender(n_components=200)
my_rec_engine.fit(ratings_mat)
lucky_user_recommend, items_rated_by_this_user = my_rec_engine.top_n_recs(user_id=lucky_user_index, n = 10)
print("The top ten recommendation for user %s are: " % (lucky_user))
print('%s' % (', '.join(list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                       for i in lucky_user_recommend)))

The top ten recommendation for user OLGQ7alK4VKl3YdQk6UF5g are: 
Spice Finch, Wiz Kid, Green Basil Thai Kitchen, Mumbai Bistro, Tria Cafe Wash West, Marrakesh, Octopus Falafel Truck, Suraya, Han Dynasty, HipCityVeg - University City


In [ ]:
print("The users original rated resturants are :\n %s" % (','.join(list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                       for i in items_rated_by_this_user)))





The users original rated resturants are :
 R2L,Vetri Cucina,Prohibition Taproom,Marmont Steakhouse & Bar,Village Whiskey,Wawa


In [ ]:
original_rated_restaurants = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] for i in items_rated_by_this_user]
mask = [name in original_rated_restaurants for name in df['name']]
original_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
original_category_vec = vectorizer.fit_transform(original_category).toarray()
original_word = vectorizer.get_feature_names_out()
print('Categories from user rated restaurants: \n%s' % (','.join(i for i in original_word)))

Categories from user rated restaurants: 
american,automotive,bar,bars,beer,breakfast,brunch,burgers,cocktail,coffee,convenience,cuban,delis,food,gas,gastropubs,italian,lounges,new,nightlife,plates,restaurants,sandwiches,seafood,small,stations,steakhouses,stores,tapas,tea,traditional,whiskey


In [ ]:
recommend_res = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                 for i in lucky_user_recommend]
mask = [name in recommend_res for name in df['name']]
recommend_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
recommend_category_vec = vectorizer.fit_transform(recommend_category).toarray()
recommend_word = vectorizer.get_feature_names_out()
print('Categories from recommend restaurants: \n%s' % (','.join(i for i in recommend_word)))

Categories from recommend restaurants: 
active,american,bars,beer,breakfast,brunch,burgers,cafes,cheese,cheesesteaks,chinese,coffee,desserts,eastern,european,falafel,fast,food,gardens,grocery,indian,lebanese,life,mediterranean,middle,modern,moroccan,new,nightlife,parks,restaurants,sandwiches,shops,specialty,spirits,stands,szechuan,tea,thai,trucks,vegan,wine


In [ ]:
#Check the common labels
print("Common labels are: \n%s" % (', '.join(word for word in recommend_word if word in original_word)))

Common labels are: 
american, bars, beer, breakfast, brunch, burgers, coffee, food, new, nightlife, restaurants, sandwiches, tea


In [ ]:
#Matrix Factorization recommender (SVD) with restaurants' labels.
#get the number of labels 
mask = [business in business_id for business in df['business_id']]
category = df['categories'][mask]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
category_vec = vectorizer.fit_transform(category).toarray()
words = vectorizer.get_feature_names_out()
#This is the number of unique categories
print('The total number of restaurant labels is %d' % (len(words))) 

The total number of restaurant labels is 167


In [ ]:
from sklearn.decomposition import TruncatedSVD
class SVD_Recommender(object):

    def __init__(self):
        self.n_components = 160 #the number of labels

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        svd = TruncatedSVD(n_components=self.n_components, n_iter=7, random_state=1)
        svd.fit(ratings_mat)
        self.V = svd.components_
        self.U = svd.transform(ratings_mat)
        self.ratings_mat_fitted = self.U.dot(self.V)

    def get_error(self):
        return ((self.ratings_mat_fitted - self.ratings_mat)**2).mean(axis=None)
        
    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        cleaned_out = self.ratings_mat_fitted[user_id,:]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return cleaned_out


    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n):
        pred_ratings = self.pred_one_user(user_id)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:], items_rated_by_this_user

In [ ]:
# get recommendations for the same lucky user
my_rec_engine = SVD_Recommender()
my_rec_engine.fit(ratings_mat)
lucky_user_recommend, items_rated_by_this_user = my_rec_engine.top_n_recs(user_id=lucky_user_index, n = 10)
print("The top ten recommendation for user %s are: " % (lucky_user))
print('%s' % (', '.join(list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                       for i in lucky_user_recommend)))

The top ten recommendation for user OLGQ7alK4VKl3YdQk6UF5g are: 
Honey's Sit-N-Eat, HipCityVeg - University City, Wawa, O Sole Mio, Boostin' Bowls, Red Owl Tavern, Golden Chopstick Chinese Restaurant, Hibachi 2 Go, Trio Breakfast & Lunch, Dagwood's Pub


In [ ]:
original_rated_restaurants = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] for i in items_rated_by_this_user]
mask = [name in original_rated_restaurants for name in df['name']]
original_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
original_category_vec = vectorizer.fit_transform(original_category).toarray()
original_word = vectorizer.get_feature_names_out()
print('Categories from user rated restaurants: \n%s' % (','.join(i for i in original_word)))

Categories from user rated restaurants: 
american,automotive,bar,bars,beer,breakfast,brunch,burgers,cocktail,coffee,convenience,cuban,delis,food,gas,gastropubs,italian,lounges,new,nightlife,plates,restaurants,sandwiches,seafood,small,stations,steakhouses,stores,tapas,tea,traditional,whiskey


In [ ]:
recommend_res = [list(set(df['name'][df['business_id'] == business_id[i]]))[0] \
                 for i in lucky_user_recommend]
mask = [name in recommend_res for name in df['name']]
recommend_category = df['categories'][mask]
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True
                            )
recommend_category_vec = vectorizer.fit_transform(recommend_category).toarray()
recommend_word = vectorizer.get_feature_names_out()
print('Categories from recommend restaurants: \n%s' % (','.join(i for i in recommend_word)))

Categories from recommend restaurants: 
acai,american,automotive,bars,bowls,breakfast,brunch,burgers,chinese,coffee,convenience,delis,desserts,fast,food,gas,gelato,italian,japanese,juice,new,nightlife,pizza,pubs,restaurants,sandwiches,sardinian,seafood,smoothies,southern,stations,stores,sushi,tea,traditional,vegan


In [ ]:
#Check the common labels
print("Common labels are: \n%s" % (', '.join(word for word in recommend_word if word in original_word)))

Common labels are: 
american, automotive, bars, breakfast, brunch, burgers, coffee, convenience, delis, food, gas, italian, new, nightlife, restaurants, sandwiches, seafood, stations, stores, tea, traditional
